## Setup

In [1]:
!pip install numpy ortools
import numpy as np


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


## Um algoritmo de força bruta pode ser representado pelos seguintes passos:

<ol>
<li>Defina uma variável max (min) como -infinita (infinita)</li>
<li>Defina seu vetor de variáveis candidatas</li>
<li>Delimite o espaço de buscas através das restrições</li>
<li>No espaço de buscas, para cada combinação possível de valores das variáveis que respeite as restrições: </li>
  <ul>
    <li>Aplique a combinação de valores na função objetivo</li>
    <li>Se o valor da combinação de valores aplicada função objetivo for \( \gt \text{max} \) </li>
    <ul>
      <li>Atualize o vetor de variáveis candidatas com a combinação de valores</li>
      <li>Atualize max para o valor da aplicação da combinação de valores na função objetivo</li>
    </ul>
  </ul>
  <li>Ao término do algoritmo é garantido que é encontrado o vetor ótimo de variáveis que maximiza(minimiza) a função objetivo</li>
</ol>

### Problema das namoradas ser solucionado:

$$
  \text{max}(L + G)
$$

Sujeito a:

$$    
  180L + 100G \leq 800 \text{ (Restrição monetária) }\\
  2L + 4G \leq 20 \text{ (Restrição de tempo) }\\
  L, G \in Z_{+}
$$

### Passo 1: Defina máximo como -infinito

In [2]:
infinito = np.inf
maximo = -infinito

### Passo 2: Defina seu conjunto de variáveis candidatas

In [3]:
L_cand, G_cand = 0, 0

### Passo 3: Delimite o espaço de buscas através das restrições

Como fazer isso?

Considerando que temos $n$ variáveis de decisão no nosso problema e cada uma delas é uma variável $x_i$.
Considerando também que temos $m$ restrições no nosso problema.

Para cada variável i do nosso sistema e para cada restrição j do nosso sistema devemos:

<ol>
    <li>Dividir o termo independente da restrição $j: \beta_j$ pelo coeficiente $c_{ji}$ da variável $x_i$</li>
    <li>Escolher como limite superior da variável $x_i$ a maior razão entre $\beta_j \over c_{ji}$</li>
</ol>

Especificado pelo pseudo codigo abaixo:

```
inteiro limite_superior[n];
para i := 0 i < n; faça
    razao := -infinito
    para j := 0 < m; faça
        se beta[j] / c[j][i] > razao entao
            razao := beta[j] / c[j][i]
            limite_superior[i] := razao
        fimse
    fimpara
fimpara
```

### Considerando nosso problema 

$$
  \text{max}(L + G)
$$

Sujeito a:

$$    
  180L + 100G \leq 800 \text{ (Restrição monetária) }\\
  2L + 4G \leq 20 \text{ (Restrição de tempo) }\\
  L, G \geq 0 \in Z
$$

temos duas restrições, logo:

In [4]:
limite_superior = [None, None]
matriz_restricoes = np.array(
    [[180, 100],
     [2,     4]]
)
beta = np.array(
    [800, 20]
)
quantidade_variaveis = 2
quantidade_restricoes = 2
for i in range(quantidade_variaveis):
    razao = -infinito
    for j in range(quantidade_restricoes):
        if beta[j] // matriz_restricoes[j][i] > razao:
            razao = beta[j] // matriz_restricoes[j][i]
            limite_superior[i] = razao
            
limite_superior_L = limite_superior[0]
limite_superior_G = limite_superior[1]
print(limite_superior_L, limite_superior_G)

10 8


### Passo 4 (explorando o espaço de buscas)

No espaço de buscas, para cada combinação possível de valores das variáveis que respeite as restrições: </li>
<ul>
  <li>Aplique a combinação de valores na função objetivo</li>
  <li>Se o valor da combinação de valores aplicada na função objetivo for \( \gt \text{max} \) </li>
  <ul>
    <li>Atualize o vetor de variáveis candidatas com a combinação de valores</li>
    <li>Atualize max para o valor da aplicação da combinação de valores na função objetivo</li>
  </ul>
</ul>

In [5]:
while L_cand <= limite_superior_L:
    G_cand = 0
    while G_cand <= limite_superior_G:
        # para cada combinacao de valores possiveis das variaveis
        if 180*L_cand + 100*G_cand <= 800 and 2*L_cand + 4*G_cand <= 20:
            # que respeite as restrições do problema
            objetivo = L_cand + G_cand # aplico a combinação de valores na funcao objetivo
            
            if objetivo > maximo: # se a nova combinacao for maior do que o maximo atual
                G, L = G_cand, L_cand # atualizo as variaveis candidatas
                maximo = objetivo # atualizo o valor maximo
        G_cand += 1
    L_cand += 1
Z = maximo

### Passo 5: Otimização encontrada com valores das variáveis de decisão pela força bruta

In [6]:
print('Z: {}\nL: {}\nG: {}'.format(Z, L, G))

Z: 6
L: 2
G: 4


### Passo 6: Comparando os valores com os encontrados pelo solver do google

In [8]:
# define solver de programação inteira
from ortools.linear_solver import pywraplp
solver = pywraplp.Solver.CreateSolver('SAT')
# adiciona variaveis de decisao
L_solver = solver.IntVar(0, infinito, 'L_solver')
G_solver = solver.IntVar(0, infinito, 'G_solver')
# adiciona restricoes
solver.Add(180 * L_solver + 100 * G_solver <= 800)
solver.Add(2 * L_solver + 4 * G_solver <= 20)
# adiciona objetivo
solver.Maximize(L_solver + G_solver)
# resolve
status = solver.Solve()
# confere se existe solucao
if status == 0:
    # confere se a solucao obtida pela forca bruta é a mesma do solver do google
    L_solver = L_solver.solution_value()
    G_solver = G_solver.solution_value()
    Z_solver = solver.Objective().Value()
    if (Z, L, G) == (Z_solver, L_solver, G_solver):
        print('A solução da força bruta é a mesma do solver do google')

A solução da força bruta é a mesma do solver do google
